<a href="https://colab.research.google.com/github/sanika-tikole/InnomaticInternshipProject2/blob/main/emailautomation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit pyngrok pandas openpyxl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.6 MB/s eta 0:00:00


In [2]:
code = '''
import streamlit as st
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import ssl

def send_email(name, email, subject, body, sender_email, sender_password, pdf_file=None):
    try:
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = email
        msg['Subject'] = subject
        msg.attach(MIMEText(body, 'plain'))

        if pdf_file:
            part = MIMEBase('application', 'octet-stream')
            part.set_payload(pdf_file.read())
            encoders.encode_base64(part)
            part.add_header('Content-Disposition', f'attachment; filename="{pdf_file.name}"')
            msg.attach(part)

        context = ssl.create_default_context()
        with smtplib.SMTP("smtp.gmail.com", 587) as server:
            server.starttls(context=context)
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, email, msg.as_string())

        return f"✅ Sent to {name} ({email})"
    except Exception as e:
        return f"❌ Failed to {name}: {str(e)}"

st.set_page_config(page_title="AI Email Agent", layout="centered")
st.sidebar.title("📬 AI Email Agent")
section = st.sidebar.radio("Choose Function:", ["Marketing Email Sender", "Interview Meeting Email Sender", "Custom Bulk Email Sender"])

st.header(f"📌 {section}")
sender_email = st.text_input("Sender Email", placeholder="you@gmail.com")
sender_password = st.text_input("App Password (Gmail)", type="password")
excel_file = st.file_uploader("Upload Excel File", type=["xlsx"])
pdf_file = st.file_uploader("📎 Optional: Upload PDF", type=["pdf"])
email_subject = st.text_input("Subject Line", value=" !")

contacts = None
if excel_file:
    try:
        contacts = pd.read_excel(excel_file)
        if 'Name' not in contacts.columns or 'Email' not in contacts.columns:
            st.error("Excel must have 'Name' and 'Email' columns")
            contacts = None
    except Exception as e:
        st.error(f"Error reading Excel file: {e}")

if section == "Marketing Email Sender":
    st.subheader("📣 Compose Marketing Email")
    email_intro = "Hi {name},\\n\\nWe’re excited to share our latest product with you!\\n"
    email_footer = "\\n\\nFollow us:\\nLinkedIn: {website}\\nInstagram: {instagram}"
    user_message = st.text_area("📝 Add your custom content", height=150)
    website = st.text_input("website Link")
    instagram = st.text_input("Instagram Link")
    full_body = email_intro + user_message + email_footer

elif section == "Interview Meeting Email Sender":
    st.subheader("📄 Compose Interview Email")
    email_intro = "Dear {name},\\n\\nYou have been shortlisted for an interview for {job_role} position.\\n"
    email_footer = "\\n\\nPlease be on time.\\nBest regards,\\nHR Team"
    job_role = st.text_input("Job Role")
    meeting_link = st.text_input("Zoom/Meet Link")
    time_slot = st.text_input("Scheduled Time")
    user_message = st.text_area("📋 Interview Details", height=150)
    full_body = email_intro + user_message + f"\\n\\nMeeting Link: {meeting_link}\\nTime: {time_slot}" + email_footer

elif section == "Custom Bulk Email Sender":
    st.subheader("🛠️ Write Your Full Message")

    user_message = st.text_area(
        "Enter your custom message below. The greeting 'Dear {name},' will be added automatically.",
        height=250
    )

    # Add standard greeting
    email_intro = """Dear {name},

"""
    full_body = email_intro + user_message

if st.button("🚀 Send Emails"):
    if not all([sender_email, sender_password, contacts is not None, full_body]):
        st.warning("Please fill in all required fields.")
    else:
        logs = []
        for _, row in contacts.iterrows():
            name = row['Name']
            email = row['Email']
            try:
                formatted_body = full_body.format(
                    name=name,
                    website=website if section == "Marketing Email Sender" else "",
                    instagram=instagram if section == "Marketing Email Sender" else "",
                    job_role=job_role if section =="Interview Meeting Email Sender" else""
                )
            except KeyError as e:
                st.error(f"Missing placeholder value: {e}")
                break
            result = send_email(name, email, email_subject, formatted_body, sender_email, sender_password, pdf_file)
            logs.append(result)
        st.success("✅ Email process complete!")
        for log in logs:
            st.write(log)
'''
with open("app.py", "w") as f:
    f.write(code)

In [3]:
!ngrok config add-authtoken 2yGoKhUCLFBfu8kO3lfosPkZvEn_CdpzBrjUuo5SmxCSq3od


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
from pyngrok import ngrok

# Start Streamlit app
!streamlit run app.py &>/content/logs.txt &

# Open ngrok tunnel the correct way (updated)
public_url = ngrok.connect(addr="8501", proto="http")
print(f"🌐 Your app is live at: {public_url}")


🌐 Your app is live at: NgrokTunnel: "https://d6f6-34-34-5-31.ngrok-free.app" -> "http://localhost:8501"
